## 利用飞桨PaddleSeg进行遥感影像地块分割

### PaddleSeg介绍
- PaddleSeg是基于飞桨PaddlePaddle开发的端到端图像分割开发套件，涵盖了高精度和轻量级等不同方向的大量高质量分割模型。通过模块化的设计，提供了配置化驱动和API调用两种应用方式，帮助开发者更便捷地完成从训练到部署的全流程图像分割应用。


第一步：新建notebook，选择通用

第二步：配置notebook

- 开发语言选择Python3.7，AI框架选择PaddlePaddle 2.0.0，资源规格选择GPU V100，启动。打开Notebook。
- 上传本操作模板

第三步：安装环境

- 环境准备：安装paddlex，paddlepaddle-gpu。

In [ ]:
# 升级pip
!pip install --upgrade pip -i https://mirror.baidu.com/pypi/simple

In [ ]:
!python -m pip install paddlepaddle-gpu==2.1.3.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

In [ ]:
!pip install paddlex==2.0.0 -i https://mirror.baidu.com/pypi/simple

第四步：下载代码包
- 上传提前下载好的[PaddleSeg代码包](https://github.com/PaddlePaddle/PaddleSeg/)。
- PaddleSeg2.3版本
- 解压代码包

In [ ]:
# 进入work
%cd /home/aistudio/work

# 下载PaddleSeg2.3
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git -b release/2.3

In [ ]:
# 安装paddleseg
!pip install paddleseg

第五步：数据集上传并解压

- 回到根目录
- 上传数据集
- 解压数据集

In [ ]:
# 解压训练数据
!unzip -q /home/aistudio/data/data77571/train_and_label.zip -d /home/aistudio/data/BlockSeg

!unzip -q /home/aistudio/data/data77571/img_test.zip -d /home/aistudio/data/BlockSeg

第六步：数据切分
- 数据切分：
- 若显示无filelock，则先执行下面命令安装filelock，再执行数据切分命令


In [ ]:
! pip install filelock -i https://mirror.baidu.com/pypi/simple

In [ ]:
# 划分训练集、测试集，比例为8：2

import os
import random
import shutil
from tqdm import tqdm

label_path = "/home/aistudio/data/BlockSeg/lab_train"
img_path = "/home/aistudio/data/BlockSeg/img_train"
save_path = "/home/aistudio/data/BlockSeg"

train_img_path = os.path.join(save_path, "train", "images")
train_ann_path = os.path.join(save_path, "train", "annotations")

val_img_path = os.path.join(save_path, "val", "images")
val_ann_path = os.path.join(save_path, "val", "annotations")

os.makedirs(train_img_path)
os.makedirs(train_ann_path)
os.makedirs(val_img_path)
os.makedirs(val_ann_path)

name_list = os.listdir(img_path)
# print(name_list)
random.shuffle(name_list)

train_list_path = os.path.join(save_path, "train.txt")
val_list_path = os.path.join(save_path, "val.txt")
train_file = open(train_list_path, "a")
val_file = open(val_list_path, "a")

for name in tqdm(name_list[:int(len(name_list)*0.8)]):
    img_old_path = os.path.join(img_path, name)
    img_new_path = os.path.join(train_img_path, name)
    shutil.copy(img_old_path, img_new_path)

    ann_old_path = os.path.join(label_path, name.replace("jpg", "png"))
    ann_new_path = os.path.join(train_ann_path, name.replace("jpg", "png"))
    shutil.copy(ann_old_path, ann_new_path)
    
    train_file.write("train/images/{} train/annotations/{}\n".format(name, name.replace("jpg", "png")))

for name in tqdm(name_list[int(len(name_list)*0.8):]):
    img_old_path = os.path.join(img_path, name)
    img_new_path = os.path.join(val_img_path, name)
    shutil.copy(img_old_path, img_new_path)

    ann_old_path = os.path.join(label_path, name.replace("jpg", "png"))
    ann_new_path = os.path.join(val_ann_path, name.replace("jpg", "png"))
    shutil.copy(ann_old_path, ann_new_path)

    val_file.write("val/images/{} val/annotations/{}\n".format(name, name.replace("jpg", "png")))

In [ ]:
# 查看数据集目录

!tree /home/aistudio/data/BlockSeg/ -L 1


第七步：模型训练

- 修改目录PaddleSeg/configs/segformer/segformer_b0_cityscapes_1024x1024_160k.yml文件中的以下配置：
-- batch_size：8
-- iters：80000
-- model下的num_classes：5


- 修改PaddleSeg-release-2.3/confligs/_base_/cityscapes.yml配置,直接改为以下配置

```
batch_size: 8
iters: 80000

train_dataset:
  type: Dataset
  dataset_root: /home/aistudio/data/BlockSeg
  train_path: /home/aistudio/data/BlockSeg/train.txt
  num_classes: 5
  transforms:
    - type: ResizeStepScaling
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop
      crop_size: [1024, 512]
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/aistudio/data/BlockSeg
  val_path: /home/aistudio/data/BlockSeg/val.txt
  num_classes: 5
  transforms:
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.01
  end_lr: 0
  power: 0.9

loss:
  types:
    - type: CrossEntropyLoss
  coef: [1]
```

- 进入PaddleSeg工作目录

In [ ]:
%cd /home/aistudio/work/PaddleSeg

- 选择启动单GPU卡训练模型

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 # 设置1张可用的卡

- 启动模型训练
-- 选择网络结构配置
-- do_eval:每训练完一次都要进行一次验证
-- use_vdl:可以启用vdl进行可视化训练
-- save_intervel：每经过2000liters保存一次模型
-- save_dir output：保存模型的路径在output文件夹内

In [ ]:
#!python train.py --config configs/segformer/segformer_b0_cityscapes_1024x1024_160k.yml --do_eval --use_vdl --save_interval 50 --save_dir output

!python train.py --config configs/fastscnn/fastscnn_cityscapes_1024x1024_160k.yml --do_eval --use_vdl --save_interval 50 --save_dir output

In [13]:
# 断点续训
#!python train.py --config configs/segformer/segformer_b0_cityscapes_1024x1024_160k.yml --resume_model output/iter_115000 --do_eval --use_vdl --save_interval 5000 --save_dir output

!python train.py --config configs/fastscnn/fastscnn_cityscapes_1024x1024_160k.yml --resume_model output/iter_5000 --do_eval --use_vdl --save_interval 5000 --save_dir output

2021-11-30 09:37:56 [INFO]	[TRAIN] epoch: 2, iter: 11530/160000, loss: 1.7873, lr: 0.046752, batch_cost: 1.0802, reader_cost: 0.91451, ips: 7.4060 samples/sec | ETA 44:32:58
2021-11-30 09:38:07 [INFO]	[TRAIN] epoch: 2, iter: 11540/160000, loss: 1.6959, lr: 0.046749, batch_cost: 1.1210, reader_cost: 0.95655, ips: 7.1365 samples/sec | ETA 46:13:43
^C


- 第八步：模型验证

In [ ]:
!python val.py --config configs/segformer/segformer_b0_cityscapes_1024x1024_160k.yml --model_path output/best_model/model.pdparams

在图像分割领域中，评估模型质量主要是通过三个指标进行判断，准确率（acc）、平均交并比（Mean Intersection over Union，简称mIoU）、Kappa系数。

- 准确率：指类别预测正确的像素占总像素的比例，准确率越高模型质量越好。
- 平均交并比：对每个类别数据集单独进行推理计算，计算出的预测区域和实际区域交集除以预测区域和实际区域的并集，然后将所有类别得到的结果取平均。在本例中，正常情况下模型在验证集上的mIoU指标值会达到0.80以上，显示信息示例如下所示，第3行的mIoU=0.8526即为mIoU。
- Kappa系数：一个用于一致性检验的指标，可以用于衡量分类的效果。kappa系数的计算是基于混淆矩阵的，取值为-1到1之间，通常大于0。其公式如下所示，P0P_0P0为分类器的准确率，PeP_eP**e为随机分类器的准确率。Kappa系数越高模型质量越好。

- 第九步：模型预测
-- 输出的预测结果，可在PaddleSeg-release-2.3/output/result/added_prediction 内查看

In [15]:
#!python predict.py --config configs/segformer/segformer_b0_cityscapes_1024x1024_160k.yml --model_path output/iter_130000/model.pdparams --image_path /home/aistudio/data/BlockSeg/img_testA --save_dir output/result

!python predict.py --config configs/fastscnn/fastscnn_cityscapes_1024x1024_160k.yml --model_path output/iter_10000/model.pdparams --image_path /home/aistudio/data/BlockSeg/img_testA --save_dir output/result

2021-11-30 09:38:47 [INFO]	
---------------Config Information---------------
batch_size: 8
iters: 160000
loss:
  coef:
  - 1.0
  - 0.4
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0.0001
  learning_rate: 0.05
  power: 0.9
  type: PolynomialDecay
model:
  enable_auxiliary_loss: true
  num_classes: 5
  pretrained: null
  type: FastSCNN
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: /home/aistudio/data/BlockSeg
  mode: train
  num_classes: 5
  train_path: /home/aistudio/data/BlockSeg/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 1024
    - 1024
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_range: 0.4
    contrast_range: 0.4
    saturation_range: 0.4
    type: RandomDistort
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: /home/aistudio/data/BlockSeg
  mode: val
  num_classes: 5
  t

- 第十步：输出比赛结果
-- 输出的预测结果，可在PaddleSeg-release-2.3/output/result/added_prediction 内查看

In [16]:
%cd output/result

/home/aistudio/work/PaddleSeg/output/result


In [17]:
!zip -q -r result.zip pseudo_color_prediction
